In [2]:
const ○ = 1 # 𝟘 ○ OF
const ● = 2 # 𝟙 ● ON

2

In [61]:
state_count = 4

# state by state flat 
p = 
Float64[
#              ○ ● ○ ● ○ ● ○ ● ○ ● ○ ● ○ ● ○ ●
#              ○ ○ ● ● ○ ○ ● ● ○ ○ ● ● ○ ○ ● ●
#              ○ ○ ○ ○ ● ● ● ● ○ ○ ○ ○ ● ● ● ●
#              ○ ○ ○ ○ ○ ○ ○ ○ ● ● ● ● ● ● ● ●    
#= ○ ○ ○ ○ =#  1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
#= ● ○ ○ ○ =#  0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0;
#= ○ ● ○ ○ =#  0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0;
#= ● ● ○ ○ =#  0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
#= ○ ○ ● ○ =#  0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
#= ● ○ ● ○ =#  0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0;
#= ○ ● ● ○ =#  0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0;
#= ● ● ● ○ =#  0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0;
#= ○ ○ ○ ● =#  0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0;
#= ● ○ ○ ● =#  0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0;
#= ○ ● ○ ● =#  0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0;
#= ● ● ○ ● =#  0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0;
#= ○ ○ ● ● =#  0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0;
#= ● ○ ● ● =#  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1;
#= ○ ● ● ● =#  0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0;
#= ● ● ● ● =#  0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0;
]
q = reshape(p, (ntuple(x->2, state_count*2)));
length(size(q))

8

In [6]:
# candidate system ABC fixing D
q2 = q[:,:,:,○,:,:,:,:] # fixing background condition D at t
q3 = q2[:,:,:,:,:,:,○] + q2[:,:,:,:,:,:,●] # marganalizing out state D at t+1
q4 = q3 ./ sum(q3, dims=(4,5,6)) # normailizing rows
reshape(q4, 8, 8) 

8×8 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0

In [28]:
# Mechanism C over purview BC

# Feed in a uniform distribution to A's inputs

q5 = q4[:,:,:,:,○,:] + q4[:,:,:,:,●,:] # marganilize out B at t+1
q6 = q5 ./ sum(q5, dims=(4,5)) # normailizing rows

q7 = q6[:,:,:,:,○] + q6[:,:,:,:,●] # marganilize out C at t + 1
q8 = q7 ./ sum(q7, dims=(4)) # normailizing rows

q9 = q8[○,:,:,:] + q8[●,:,:,:] # marganilize out A at t - 1
q10 = q9 ./ sum(q9, dims=(3)) # normailizing rows

q11 = q10[○,:,:] + q10[●,:,:] # marganilize out B at t - 1
q12 = q11 ./ sum(q11, dims=(2)) # normailizing rows

q13 = q12[○,:] + q12[●,:] # marganilize out C at t - 1
q14 = q13 ./ sum(q13, dims=(1)) # normailizing rows


2-element Vector{Float64}:
 0.25
 0.75

In [104]:
mutable struct TPM
    tpm
    current_elements # element to tpm dimension index
    next_elements # element to tpm dimension index
end


function make_tpm(flat, elements)
    @assert size(flat) == (length(elements)^2, length(elements)^2)
    
    TPM(
        reshape(flat, (ntuple(x->2, length(elements)*2))),
        Dict([(value, index) for (index, value) in enumerate(elements)]),
        Dict([(value, index + length(elements)) for (index, value) in enumerate(elements)])
    )
end

function adjust_dims(elements, removed_dim)

    Dict([dim > removed_dim ? (element, dim-1) : (element, dim) for (element, dim) in elements])
end

function flatten_tpm(tpm)
    reshape(tpm.tpm, 2^length(tpm.current_elements), 2^length(tpm.next_elements)) 
end

function fix_condition(tpm, element, current, value) 
    @assert value == ○ || value == ●
    
    dim = current ? tpm.current_elements[element] : tpm.next_elements[element]
    tpm2 = tpm.tpm[ntuple(x->Colon(), dim-1)..., value, ntuple(x->Colon(), len-dim)...]
    
    current_elements = adjust_dims(tpm.current_elements, dim)
    next_elements = adjust_dims(tpm.next_elements, dim)

    delete!(current ? current_elements : next_elements, element)
    
    TPM(tpm2, current_elements, next_elements)
end 

function marganilize(tpm, element, current) 
    len = length(size(tpm.tpm))
    
    dim = current ? tpm.current_elements[element] : tpm.next_elements[element]
    
    current_count = length(tpm.current_elements)
   
    tpm2 = tpm.tpm[ntuple(x->Colon(), dim-1)..., ○, ntuple(x->Colon(), len-dim)...] + tpm.tpm[ntuple(x->Colon(), dim-1)..., ●, ntuple(x->Colon(), len-dim)...]
    tpm3 = tpm2 ./ sum(tpm2, dims=(current ? [current_count:len-1...] : [current_count+1:len-1...])) # normailizing rows
    
    current_elements = adjust_dims(tpm.current_elements, dim)
    next_elements = adjust_dims(tpm.next_elements, dim)
    
    delete!(current ? current_elements : next_elements, element)

    TPM(tpm3, current_elements, next_elements)
end 

marganilize (generic function with 1 method)

In [108]:
tpm_a = make_tpm(p, [:a, :b, :c, :d])
tpm_a = fix_condition(tpm_a, :d, true, ○)
tpm_a = marganilize(tpm_a, :d, false)
tpm_a = marganilize(tpm_a, :b, false)
tpm_a = marganilize(tpm_a, :c, false)
tpm_a = marganilize(tpm_a, :a, true)
tpm_a = marganilize(tpm_a, :b, true)
tpm_a = marganilize(tpm_a, :c, true)
flatten_tpm(tpm_a)

1×2 Matrix{Float64}:
 0.25  0.75

In [55]:
t_states = 4
m=9
len = length(size(q))
t_1_states = len - t_states

q2 = q[ntuple(x->Colon(), m-1)..., ○, ntuple(x->Colon(), len-m)...] + q[ntuple(x->Colon(), m-1)..., ●, ntuple(x->Colon(), len-m)...]
q2 ./ sum(q2, dims=(m > t_states ? [t_states+1:len-1...] : [t_states:len-1...])) # normailizing rows

reshape(q2, 16, 8)

LoadError: AssertionError: m > 0 && m <= len

In [59]:
length([1,1,1])

3

In [60]:
:a

:a